In [1]:
import arcpy
import os
import numpy as np
import pandas as pd

# Input Parameters 

In [2]:
arcpy.env.overwriteOutput = True
arcpy.env.workspace='F:\SW023129 Los Pen WMP\900-GIS\930-Flood Control\932 Delineation\932.1 Carroll Canyon\932.1.1 East of 15\QGIS_Sandbox\GDB.gdb'
# catchment = 'catchment'
catchment = 'upstream_s24745_catchment'
drainageline = 'ArcHydro_DL_Clip'
dl_cat_int = 'dl_cat_int_all'

cat_usid = 'new_ID_S'
cat_dsid = 'new_DwnID_S' #to be filled here
dl_usid = 'CatID'
dl_dsid = 'DwnCatID'

# Helper Function

In [26]:
def read_at(input_data):
    #Convert the attribute table of a feature class to pandas database
    arcpy.conversion.TableToExcel(Input_Table=input_data,
                              Output_Excel_File='Output/_temp_input.xls'
                             )
    df_at = pd.read_excel('Output/_temp_input.xls')
    os.remove('Output/_temp_input.xls')
    return df_at

def Con_in_cat(df_dl):
    df_dl_id_unique = df_dl.drop_duplicates(subset=[dl_usid,dl_dsid])
    max_row = 0
    ds_of_max = []
    sr_us = pd.Series([])
    for index,row in df_dl_id_unique.iterrows():
        _us_list = trace_upstream(row[dl_usid],df_dl_id_unique)
        if _us_list.size == 1:
            sr_us = sr_us.append(_us_list.iloc[0])
        if _us_list.size>max_row:
            max_row = _us_list.size
            df_lgst_flpth = _us_list
            ds_of_max=_us_list.iloc[-1]
    
    # convert upstream list and downstream catchment ID into a dataframe
    sr_us.reset_index(drop=True, inplace=True)
    _df_con_in_cat = pd.DataFrame(sr_us,columns=['us'])
    _df_con_in_cat['ds']=ds_of_max[0]
    return _df_con_in_cat,df_lgst_flpth

def trace_upstream(target_ID, df):
    _neighbors = df[df[dl_dsid] == target_ID]
    upstream = _neighbors
    while not _neighbors.empty:
        _neighbors = df[df[dl_dsid].isin(_neighbors[dl_usid])]
        upstream = upstream.append(_neighbors, ignore_index=True)
    #Comment out the line below if one does not want to include the catchment itself
    upstream=upstream.append(df[df[dl_usid]==target_ID],ignore_index=True)
    exc_cols=df.columns.difference([dl_usid])
    upstream = upstream.drop(exc_cols,1)
    return upstream

def lgst_flpth(df_flpth,org_shp,catID=cat_usid, dlID=dl_usid):
     # Define output variables
#     target_ID = df_fpth[catID][0]
    temp_input='_temp_input_lgst_flpth'
    temp_input_xls='Output/'+temp_input+'.xls'
    table_gdb='a_lgst_flpth'
    table_csv=filename = 'Output/'+table_gdb+'.csv'
    fc_output='lgs_flpth'
    
    df_flpth.to_csv(table_csv,index=False)
    arcpy.conversion.TableToTable(table_csv,
                                 arcpy.env.workspace,
                                table_gdb)
    
    #Create a temp duplicate shapefile in prep for join field - avoid modifying the original input
    arcpy.conversion.FeatureClassToFeatureClass(org_shp, 
                                            arcpy.env.workspace, 
                                            temp_input)
    
    arcpy.management.JoinField(in_data=temp_input,
                          in_field=dlID,
                          join_table= table_gdb,
                          join_field=dlID)
    
    expression = dlID+'_1 IS NOT NULL' #Furture improvement: Avoid hard coding the epxression
    arcpy.conversion.FeatureClassToFeatureClass(temp_input, 
                                            arcpy.env.workspace, 
                                            fc_output,
                                           expression)
    arcpy.management.Delete(temp_input)
    arcpy.management.DeleteField(fc_output,dlID+'_1')

# Main Running Block 

In [5]:
# Intersect drainage line by catchment 
arcpy.analysis.Intersect(in_features=[catchment,drainageline],
                         out_feature_class=dl_cat_int,
                         join_attributes="ALL"
                        )
# Import catchment 
df_cat = read_at(catchment)
df_dl = read_at(drainageline)
df_dl_int = read_at(dl_cat_int)

### Table 1 - df_dl_con ( Global drainage line connectivity correlation)
### Table 2 - df_con_in_cats (Drainage line connectivity correlation within each catchment)
### Table 3 - df_cat_dl_cor (catchment - d/s drainage line correlation) 

In [10]:
# Import drainage line and extract drainage line connectivity correlation
df_dl_con = df_dl[[dl_usid,dl_dsid]]
df_dl_con.set_index(dl_usid,inplace=True)

#Initialize dataframe 
df_con_in_cats = pd.DataFrame([],columns=['us','ds'])
df_cat_dl_cor = pd.DataFrame([],columns=[cat_usid,dl_usid])
df_lgst_flpth = pd.DataFrame([],columns=[cat_usid,dl_usid])
#Beginning of Loop
for index,row in df_cat.iterrows():
    # Find all u/s and d/s nodes of drainage line within each catchment
    sample_cat = row[cat_usid]
    print(sample_cat)
    _temp_con_in_cat,_temp_lgst_flpth = Con_in_cat(df_dl_int[df_dl_int[cat_usid]==sample_cat])
    _temp_lgst_flpth[cat_usid]=sample_cat
    df_con_in_cats = pd.concat([df_con_in_cats,_temp_con_in_cat]) 
    df_lgst_flpth = pd.concat([df_lgst_flpth,_temp_lgst_flpth]) 
    # Assign d/s drainage line ID to CatID
    df_cat_dl_cor.loc[len(df_cat_dl_cor)]=[sample_cat, _temp_con_in_cat['ds'][0]]
#End of Loop

df_cat_dl_cor.set_index(dl_usid,inplace=True)
df_con_in_cats.set_index('us', inplace = True)
df_lgst_flpth.to_clipboard()
#Create longest flowpath shapefile based on the list


# print(df_con_in_cats[0:3])
# print(df_cat_dl_cor)

CarE1017
CarE1033
CarE1133
CarE1093
CarE1029
CarE1032
CarE1086
CarE1138
CarE1026
CarE1015
CarE1122
CarE1137
CarE1128
CarE1016
CarE1031
CarE1025
CarE1139
CarE1129
CarE1018
CarE1141
CarE1030
CarE1088
CarE1140
CarE1134
CarE1087
CarE1001
CarE1027
CarE1028
CarE1125
CarE1130
CarE1105
CarE1131
      CatID  new_ID_S
0    s25243  CarE1017
1    s25416  CarE1017
2    s25487  CarE1017
3    s25519  CarE1017
4    s25510  CarE1017
5    s25450  CarE1017
6    s25365  CarE1017
7    s25547  CarE1017
8    s25366  CarE1017
9    s25425  CarE1017
10   s25560  CarE1017
11   s25677  CarE1017
12   s25569  CarE1017
13   s25673  CarE1017
14   s25751  CarE1017
15   s25844  CarE1017
16   s25757  CarE1017
17   s25957  CarE1017
18   s26035  CarE1017
19   s26043  CarE1017
20   s25744  CarE1017
21   s25769  CarE1017
22   s26016  CarE1017
23   s26165  CarE1017
24   s25740  CarE1017
25   s25756  CarE1017
26   s25788  CarE1017
27   s25737  CarE1017
28   s25750  CarE1017
29   s25730  CarE1017
..      ...       ...
92   s24

In [27]:
lgst_flpth(df_lgst_flpth, drainageline)

## Generate final output table based on Tables 1 - 3

In [17]:
#Prepare final output table based on Table 1-3

# iteration 1
_temp_t12 = df_dl_con.join(df_con_in_cats)
_temp_t12['ds'].fillna(_temp_t12[dl_dsid],inplace=True)
_temp_t123 = _temp_t12.join(df_cat_dl_cor,on='ds') #this table now correlates upstream dl id with downstream catch ID

df_final = df_cat_dl_cor.copy()
df_final = df_final.join(_temp_t123,rsuffix = '_ds',how='left')
df_final = df_final[df_final[cat_usid]!=df_final[cat_usid+'_ds']]
# df_final.to_clipboard()

# iteration 2
_temp_t122 = _temp_t12.join(df_con_in_cats,on='ds',rsuffix='_ds')
_temp_t123 = _temp_t122.join(df_cat_dl_cor,on='ds_ds')
df_final2 = df_final.join(_temp_t123,rsuffix = '_ds2',how='left')
df_final2 = df_final2.drop_duplicates([cat_usid,cat_usid+'_ds'])
df_final2[cat_usid+'_ds'].fillna(df_final2[cat_usid+'_ds2'],inplace=True)

df_final2.rename(index=str,columns={cat_usid+'_ds': cat_dsid},inplace=True)
#export df_final2 to excel for post-processing 
df_final2.to_clipboard()

# Debugging block

In [ ]:
df_con_in_cats_d = pd.DataFrame([],columns=['us','ds'])
df_cat_dl_cor_d = pd.DataFrame([],columns=[cat_usid,dl_usid])

sample_cat = 'CarE1105'
_temp_con_in_cat = Con_in_cat(df_dl_int[df_dl_int[cat_usid]==sample_cat])
#     print(_temp_con_in_cat)
# df_con_in_cats.append(_temp_con_in_cat)
df_con_in_cats_d = pd.concat([df_con_in_cats_d,_temp_con_in_cat]) 
# Assign one d/s to CatID, and find DwnID based on the drainageline connectivity correlation
#Add a new row to the new catchment dataframe, which will contain the catchment connectivity info eventually
df_cat_dl_cor_d.loc[len(df_cat_dl_cor)]=[sample_cat, _temp_con_in_cat['ds'][0]]

df_cat_dl_cor_d.set_index(dl_usid,inplace=True)
df_con_in_cats_d.set_index('us', inplace = True)

df_con_in_cats_d = df_con_in_cats_d.join(df_cat_dl_cor_d,on='ds')

print(df_con_in_cats_d)
print(df_cat_dl_cor_d)